In [1]:
import tensorflow as tf
from tensorflow import keras
from component.respiratory_tract import get_respiratory_tract_model
from utils.feature_extraction import  get_MFCCS_v2, oversample
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import glob
import tqdm
import os

In [2]:
def make_data(folder, n_mfcc, num_segment, segment_len, use_oversample=False):
    X1 = glob.glob(os.path.join(folder, 'p', '*.wav'))
    X0 = glob.glob(os.path.join(folder, 'n', '*.wav'))

    y = []
    X = []
    for file in tqdm.tqdm(X1):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(1)
        except:
            pass
    for file in tqdm.tqdm(X0):
        try:
            X.append(get_MFCCS_v2(file, n_mfccs=n_mfcc, num_segments=num_segment, segment_length=segment_len))
            y.append(0)
        except:
            pass
    X = np.array(X)
    y = np.array(y)
    if use_oversample:
        X, y = oversample(X, y)
    return X, y

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = set(np.arange(len(X)))

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        return self.X[indexes], self.y[indexes]

In [9]:
def get_model(n_mfcc, num_segment, save_path=None):
    model = get_respiratory_tract_model(input_shape=(n_mfcc*3+3, num_segment, 1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])
    return model

def train_and_test(model_name, train_folder, test_folder, n_mfcc, num_segment, segment_len, clear_cache=True):
    if clear_cache:
        get_MFCCS_v2.cache_clear()
        cached_data = glob.glob("cache/*.npy")
        for file in cached_data:
            os.remove(file)

    #build data
    X, y = make_data(train_folder, n_mfcc, num_segment, segment_len, use_oversample=True)
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
    class_weights = class_weight.compute_class_weight('balanced', np.unique(y), y)
    class_weights = {1 - i: x for i,x in enumerate(class_weights)}
    print(f"Class weights: {class_weights}")
    train_generator = DataGenerator(X_train, y_train, 8)
    val_generator = DataGenerator(X_val, y_val, 8)

    #train
    if not os.path.exists(f'logs/{model_name}'):
        os.mkdir(f'logs/{model_name}')
    model = get_model(n_mfcc, num_segment)
    history = model.fit(train_generator, steps_per_epoch=20, validation_steps=50, epochs=1000, validation_data=val_generator, class_weight=class_weights,
                        verbose=1, callbacks=[tf.keras.callbacks.ModelCheckpoint(f'pretrained_weight/{model_name}.h5', monitor="val_loss", save_weights_only=True, verbose=1, mode='min', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir=f'logs/{model_name}')])

    #test
    X_test, y_test = make_data(test_folder,
                     n_mfcc, num_segment, segment_len)
    model.load_weights(f"pretrained_weight/{model_name}.h5")
    y_pred = model.predict(X_test)
    y_pred_r = np.rint(y_pred)
    return history, classification_report(y_pred_r, y_test), roc_auc_score(y_test, y_pred)

In [10]:
n_mfccs = 39
num_segments = 50
segment_length = 2048
train_folder = 'coswara_wavs/train'
test_folder = 'coswara_wavs/test'
model_name = f'model_paper_{n_mfccs}_{num_segments}_{segment_length}'
history, report, auc_score = train_and_test(model_name, train_folder, test_folder, n_mfccs, num_segments, segment_length)

 75%|███████▌  | 222/296 [00:50<00:17,  4.21it/s]Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'


cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/train\p\s1x2Ze9iVZNf6oFZDAZ4S9x3Cky2_cough-heavy.wav


 82%|████████▏ | 244/296 [00:55<00:12,  4.27it/s]Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'


cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/train\p\uwgjET5xm8NHkfQt7LsUpV0xC1W2_cough-heavy.wav
cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/train\p\uwgjET5xm8NHkfQt7LsUpV0xC1W2_cough-shallow.wav


  1%|          | 14/2088 [00:03<07:56,  4.35it/s]Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'


...

 76%|███████▌  | 390/514 [01:38<00:32,  3.76it/s]Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'


cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\Tfvspm3rapd3ZLyAoMG36VxNQnr2_cough-heavy.wav
cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\Tfvspm3rapd3ZLyAoMG36VxNQnr2_cough-shallow.wav
cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\tiKv850hJFTmpCAvfJOmOOqkeUs1_cough-heavy.wav
cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\tiKv850hJFTmpCAvfJOmOOqkeUs1_cough-shallow.wav


 86%|████████▌ | 442/514 [01:51<00:19,  3.64it/s]Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'
Traceback (most recent call last):
  File "D:\project\papyrus\utils\feature_extraction.py", line 87, in get_MFCCS_v2
    audio = regions.samples
AttributeError: 'int' object has no attribute 'samples'


cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\Vq1h51z5x3Wp4wS2pCm6yEAZvu82_cough-heavy.wav
cant remove silence, error -> AttributeError: 'int' object has no attribute 'samples'
path coswara_wavs/test\n\Vq1h51z5x3Wp4wS2pCm6yEAZvu82_cough-shallow.wav


100%|██████████| 514/514 [02:11<00:00,  3.91it/s]


In [11]:
print(report)
print(auc_score)

              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93       553
         1.0       0.04      0.27      0.07        11

    accuracy                           0.86       564
   macro avg       0.51      0.57      0.50       564
weighted avg       0.97      0.86      0.91       564

0.5946251129177959
